In [1]:
import numpy as np
import pandas as pd
import os
import sys
import configparser as cp
import netCDF4 as nc

In [2]:
# Read the configuration file
config = cp.ConfigParser()
config.read('../config.ini')
print('Loaded in successfully')

Loaded in successfully


In [3]:
# Load in data for MUR SST pre 2020
path = config['settings']['MUR_pre2020'][1:]
path = f'..{path}'

# Load in HRCS mmm data
path2 = '../data/refined/HRCS_nc/1985-2019_mmm_observed.nc'

# Load in mur data
mur = nc.Dataset(path)

# load in HRCS data
hrcs = nc.Dataset(path2)

print(mur, hrcs)

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    dimensions(sizes): lat(35), lon(32), time(3653)
    variables(dimensions): float32 lat(lat), float32 lon(lon), int64 time(time), int16 analysed_sst(time, lat, lon)
    groups:  <class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    description: Subsetted HRCS Data
    source: Subsetted from 1985-2019_data.csv
    dimensions(sizes): lat(30), lon(24)
    variables(dimensions): float32 lat(lat), float32 lon(lon), float32 variable(lat, lon)
    groups: 


In [4]:
# Match the dimension of the two datasets, the latitude and longitude should be in the same range.
# Read the latitude and longitude values from the hrcs dataset
hrcs_lat = hrcs.variables['lat'][:]
hrcs_lon = hrcs.variables['lon'][:]

# Find the min and max of the latitude and longitude values
lat_min = hrcs_lat.min()
lat_max = hrcs_lat.max()
lon_min = hrcs_lon.min()
lon_max = hrcs_lon.max()

# Read the latitude and longitude values from the mur dataset
mur_lat = mur.variables['lat'][:]
mur_lon = mur.variables['lon'][:]

# Find the indices in the mur dataset that correspond to the hrcs ranges
lat_inds = np.where((mur_lat >= lat_min) & (mur_lat <= lat_max))[0]
lon_inds = np.where((mur_lon >= lon_min) & (mur_lon <= lon_max))[0]

# Use these indices to slice the mur dataset
mursst = mur.variables['analysed_sst'][:, lat_inds, lon_inds]

# Retrieve a list of the dates from the mur dataset
mursstdates = mur.variables['time'][:]

mur.close()

# Load hrcs to numpy array.
hrcs_sst = (hrcs.variables['variable'][:, :])
hrcs.close()

In [5]:
print(f'Time: {mursst.shape[0]}, Lat: {mursst.shape[1]}, Lon: {mursst.shape[2]}')
print(hrcs_sst.shape)

Time: 3653, Lat: 30, Lon: 24
(30, 24)


In [6]:
# Load mur mmm nc file
path3 = '../data/refined/mur_mmm.nc'
mur_mmm = nc.Dataset(path3)

# Match the dimension of the two datasets, the latitude and longitude should be in the same range.
lat_inds = np.where((mur_mmm['lat'][:] >= lat_min) & (mur_mmm['lat'][:] <= lat_max))[0]
lon_inds = np.where((mur_mmm['lon'][:] >= lon_min) & (mur_mmm['lon'][:] <= lon_max))[0]
calculatedmmmsst = mur_mmm['mmm_sst'][:, lat_inds, lon_inds]

# Retrieve a list of labels for the time dimension
calculatedmmmssttime = mur_mmm['time'][:]

# Convert to list
calculatedmmmssttime = calculatedmmmssttime.tolist()

# Convert every item to an int
calculatedmmmssttime = [str(int(i)) for i in calculatedmmmssttime]

# Close
mur_mmm.close()

print(len(calculatedmmmssttime), type(calculatedmmmssttime))
print(calculatedmmmsst.shape, type(calculatedmmmsst))
print(calculatedmmmssttime)

156 <class 'list'>
(156, 30, 24) <class 'numpy.ma.core.MaskedArray'>
['202001', '202002', '202003', '202004', '202005', '202006', '202007', '202008', '202009', '202010', '202011', '202012', '202101', '202102', '202103', '202104', '202105', '202106', '202107', '202108', '202109', '202110', '202111', '202112', '202201', '202202', '202203', '202204', '202205', '202206', '202207', '202208', '202209', '202210', '202211', '202212', '201001', '201002', '201003', '201004', '201005', '201006', '201007', '201008', '201009', '201010', '201011', '201012', '201101', '201102', '201103', '201104', '201105', '201106', '201107', '201108', '201109', '201110', '201111', '201112', '201201', '201202', '201203', '201204', '201205', '201206', '201207', '201208', '201209', '201210', '201211', '201212', '201301', '201302', '201303', '201304', '201305', '201306', '201307', '201308', '201309', '201310', '201311', '201312', '201401', '201402', '201403', '201404', '201405', '201406', '201407', '201408', '201409', 

In [7]:
# import the datetime module
import datetime

In [11]:
# For loop going through each 'time' step in mursst
number = 0

# delete all items in ./data/
for root, dirs, files in os.walk('./testfolder/'):
    for file in files:
        os.remove(os.path.join(root, file))

    
# Load the hrcs data
locationhrcs = '../data/refined/HRCS_nc/1985-2019_mmm_observed.nc'
hrcs = nc.Dataset(locationhrcs).variables['variable'][:] - 0.4

# bleaching threshold is hrcs plus one
bleachingthreshold = hrcs + 1

# Find the bleaching threshold average for the entire dataset. Mean should be found using non-nan values.
averagebleachingthreshold = np.nanmean(bleachingthreshold)
print(averagebleachingthreshold)

for day in range(83, mursst.shape[0]):
    # Initialize countsim
    contsim = True
    
    # Make numpy array for weekly heat stress
    totalheatstress = np.zeros((mursst.shape[1], mursst.shape[2]))
    
    # Read in the date from mursst
    date = datetime.datetime(2002, 6, 1) + datetime.timedelta(days=int(mursstdates[day]))
    
    # Convert date to string, with only year and month
    dateymd = date.strftime('%Y%m%d')
    
    
    for i in range(84):
        # Check to see if day-84 is less than 0, skip and set contsim to false. otherwise create dailyheatstress
        if day - 84 < 0:
            contsim = False
            break
        else:
            dailyheatstress = ((mursst[day - i, :, :] - 273.14) - bleachingthreshold)
            #print(dailyheatstress)
        
        # Replace any values that are below 1 in dailyheatstress with 0
        dailyheatstress[dailyheatstress < 0] = 0
        
        # Add dailyheatstress to totalheatstress
        totalheatstress += dailyheatstress
        
    if contsim:
        totalheatstress = (totalheatstress/7).round(2)
        # convert totalheatstress to a dataframe
        totalheatstress = pd.DataFrame(totalheatstress)
        
        # save as csv in ./data/{dateym}.csv
        totalheatstress.to_csv(f'./testfolder/{dateymd}.csv', index=False)

29.00313


In [ ]:
# delete all items in ./data/
for root, dirs, files in os.walk('./tempdata/'):
    for file in files:
        os.remove(os.path.join(root, file))